In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [4]:
dataset = 'Asset10k.xlsx'
df = pd.read_excel(dataset)

In [5]:
df

,AssetType,SerialNo,Manufacturer,ModelNbr,Refrigerant,RefrigerantQty
0,CHILLED BEAMS - PASSIVE,NaN,NaN,NaN,NaN,NaN
1,CHILLED BEAMS - PASSIVE,NaN,NaN,NaN,NaN,NaN
2,CHILLED BEAMS - PASSIVE,NaN,NaN,NaN,NaN,NaN
3,CHILLED BEAMS - PASSIVE,0,NaN,O,NaN,NaN
4,CHILLED BEAMS - PASSIVE,NaN,NaN,FRENGER,NaN,NaN
...,...,...,...,...,...,...
10148,CASSETTE SPLIT SYSTEM,329,MODULEX,PUH-4YKSA,NaN,0.0
10149,CASSETTE SPLIT SYSTEM,NaN,MODULEX,PUH-3YKA,NaN,0.0
10150,WALL MOUNTED SPLIT SYSTEM,H803 03649 NF,MITSUBISHI,FDC306HEN3,NaN,0.0
10151,WALL MOUNTED SPLIT SYSTEM,833325020,MITSUBISHI,SRC35ZG-S,NaN,0.0


In [6]:
X = df.iloc[:, 1:]
y = df['AssetType']

In [7]:
X

,SerialNo,Manufacturer,ModelNbr,Refrigerant,RefrigerantQty
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,0,NaN,O,NaN,NaN
4,NaN,NaN,FRENGER,NaN,NaN
...,...,...,...,...,...
10148,329,MODULEX,PUH-4YKSA,NaN,0.0
10149,NaN,MODULEX,PUH-3YKA,NaN,0.0
10150,H803 03649 NF,MITSUBISHI,FDC306HEN3,NaN,0.0
10151,833325020,MITSUBISHI,SRC35ZG-S,NaN,0.0


In [8]:
y.head()

0    CHILLED BEAMS - PASSIVE
1    CHILLED BEAMS - PASSIVE
2    CHILLED BEAMS - PASSIVE
3    CHILLED BEAMS - PASSIVE
4    CHILLED BEAMS - PASSIVE
Name: AssetType, dtype: object

In [9]:
special_values = ['NA', 'NULL', 'UNKNOWN', 'UNABLE TO LOCATE', 'NO ID', 'NO ACCESS', 'UNKOWN', 'NaN', 'na']

for column in X.columns:
    X[column] = X[column].replace(special_values, pd.NA)

In [10]:
# Label encoding for both X and y
label_encoder = LabelEncoder()

for column in X.select_dtypes(include=['object']).columns:
    X[column] = label_encoder.fit_transform(X[column].astype(str))

y = label_encoder.fit_transform(y.astype(str))

In [11]:
imputer = SimpleImputer(strategy='most_frequent')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

C:\ML\lib\site-packages\sklearn\impute\_base.py:49: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(array)


In [12]:
# Split the dataset into training and testing sets
X_train_sampled, X_test, y_train_sampled, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

In [13]:
# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

In [14]:
# Train the classifier on the training data
rf_classifier.fit(X_train_sampled, y_train_sampled)

RandomForestClassifier(random_state=42)

In [15]:
# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

In [16]:
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

C:\ML\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ML\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ML\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ML\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defi

In [17]:
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.6518956179222059
Confusion Matrix:
[[  2   0   2 ...   0   3   0]
 [  1   0   0 ...   0   2   0]
 [  0   0   8 ...   0   4   0]
 ...
 [  0   0   0 ...   3   0   0]
 [  1   0   2 ...   0 255   0]
 [  0   0   0 ...   0   0   0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.20      0.12      0.15        16
           1       0.00      0.00      0.00         5
           2       0.53      0.53      0.53        15
           3       0.00      0.00      0.00         3
           4       0.75      0.21      0.33        14
           6       0.41      0.83      0.55       253
           7       0.29      0.10      0.15        20
           8       0.50      0.45      0.48        11
           9       0.68      0.42      0.52        40
          10       0.60      0.39      0.48       104
          15       1.00      0.30      0.46        10
          16       0.00      0.00      0.00         1
          17       0.60      0.38  